In [1]:
import random
import cv2
import glob
import json
import os
from tqdm import tqdm
import time
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
import numpy as np
import sys
# sys.path.append("/kaggle/input/ultralytics-package-for-yolo-v8")

In [2]:
sys.path.append("./yolov7")
from models.experimental import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import check_file, check_img_size, check_requirements, \
    box_iou, non_max_suppression, scale_coords, xyxy2xywh, xywh2xyxy, set_logging, increment_path, colorstr

device="cuda:7"
half = device != 'cpu'  # half precision only supported on CUDA
yolo_model = attempt_load("yolov7_best.pt", map_location=device)  # load FP32 model
yolo_model.to(device)
stride = int(yolo_model.stride.max())  # model stride
imgsz = check_img_size(640, s=stride)  # check img_size
yolo_model.eval()
1

Fusing layers... 
IDetect.fuse


1

In [3]:
class Configs:
    conf_thres=0.3
    iou_thres=0.65
    classes=[0,1,2,3,4,5,6,7,8]
    agnostic_nms=False
    augment=False
optyolo = Configs

In [4]:
import torch
def detect_yolo(imgpth):
    dataset = LoadImages(imgpth, img_size=imgsz, stride=stride)
    old_img_w = old_img_h = imgsz
    old_img_b = 1
    for path, img, im0, vid_cap in dataset:
        img = torch.from_numpy(img)
        img = img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)
        with torch.no_grad():   # Calculating gradients would cause a GPU memory leak
            pred = yolo_model(img.to("cuda:7"), augment=optyolo.augment)[0]
        pred = non_max_suppression(pred, optyolo.conf_thres, optyolo.iou_thres, classes=optyolo.classes, agnostic=optyolo.agnostic_nms)
        for i, det in enumerate(pred):  # detections per image
            if len(det):
            # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()
                det[:,2] = det[:,2] - det[:,0]
                det[:,3] = det[:,3] - det[:,1]
                det[:,0] = det[:,0] + det[:,2]/2
                det[:,1] = det[:,1] + det[:,3]/2

        return pred[0].detach().cpu()

In [5]:
NAMES = ["x-label","y-label","diagram-tick",'line','dot','vertical_bar','scatter','horizontal_bar']
def divide_by_class(tensor,classes,confid):
    bbox_by_class = {i:[] for i in range(len(NAMES))}
    for i,c,confid in zip(range(tensor.shape[0]),classes,confid):
        bbox_by_class[int(c.item())].append(tensor[i].tolist()+[confid.item()])
    return bbox_by_class

In [6]:
all_imgs = [os.path.join("../datasets/dataset/keypoint/images/valid",x.strip()) for x in open("../valid_names.txt").readlines()]

In [7]:
import sys
sys.path.append("../OCR/EasyOCR/trainer")
sys.path.append("../OCR/EasyOCR")

In [8]:
from model import Model
from trainer.utils import AttrDict
import yaml
from trainer.utils import CTCLabelConverter, AttnLabelConverter, Averager

In [9]:
def get_config(file_path):
    with open(file_path, 'r', encoding="utf8") as stream:
        opt = yaml.safe_load(stream)
    opt = AttrDict(opt)
    if opt.lang_char == 'None':
        characters = ''
        for data in opt['select_data'].split('-'):
            csv_path = os.path.join(opt['train_data'], data, 'labels.csv')
            df = pd.read_csv(csv_path, sep='^([^,]+),', engine='python', usecols=['filename', 'words'], keep_default_na=False)
            all_char = ''.join(df['words'])
            characters += ''.join(set(all_char))
        characters = sorted(set(characters))
        opt.character= ''.join(characters)
    else:
        opt.character = opt.number + opt.symbol + opt.lang_char
    os.makedirs(f'./saved_models/{opt.experiment_name}', exist_ok=True)
    return opt

In [10]:
opt = get_config("../OCR/config_files/en_filtered_config.yaml")
device = torch.device('cpu')
if 'CTC' in opt.Prediction:
    converter = CTCLabelConverter(opt.character)
else:
    converter = AttnLabelConverter(opt.character)
opt.num_class = len(converter.character)

In [11]:
def get_id(texts,i):
    for k in texts:
        if i==k["id"]:
            return k
def get_bounding_box(points):
    xs = [p[0] for p in points]
    ys = [p[1] for p in points]
    x_min = min(xs)
    y_min = min(ys)
    x_max = max(xs)
    y_max = max(ys)
    return [x_min, y_min, x_max - x_min, y_max - y_min]
def create_square(point, k=5):
    if type(point)==dict:
        x,y = point["x"],point["y"]
    else:
        x,y =point
    half_k = k / 2.0

    # Define the corners of the square
    corners = [x - half_k, y - half_k,x + half_k, y - half_k,x + half_k, y + half_k,x - half_k, y + half_k]
    return corners
def polygon_to_bbox(polygon):
    # Create list of x and y coordinates
    x_coords = polygon[0::2]
    y_coords = polygon[1::2]

    # Compute minimum and maximum coordinates
    x_min, x_max = min(x_coords), max(x_coords)
    y_min, y_max = min(y_coords), max(y_coords)

    # Calculate the center of the bounding box
    x_center = (x_min + x_max) / 2.0
    y_center = (y_min + y_max) / 2.0

    # Calculate the width and height of the bounding box
    width = x_max - x_min
    height = y_max - y_min

    # Return the bounding box in the format [x, y, width, height]
    return [x_center, y_center, width, height]
def get_line_bboxes(v,ctype=None):
    texts = v["text"]
    polygon_info = []
    point_info = []
    for tinf in v["axes"]["x-axis"]["ticks"]:
        bbox_label = get_id(texts,tinf["id"])
        polygon_info.append(bbox_label)

    for tinf in v["axes"]["y-axis"]["ticks"]:
        bbox_label = get_id(texts,tinf["id"])
        polygon_info.append(bbox_label)
    return polygon_info


In [43]:
ocr_model = Model(opt)
statedict = torch.load("../OCR/saved_models_best64/best_accuracy.pth",map_location=device)
newstatedict = {}
for k in statedict:
    newk = k[7:]
    newstatedict[newk] =statedict[k]
ocr_model.load_state_dict(newstatedict)
ocr_model.eval()
1

No Transformation module specified


1

In [44]:
from torchvision import transforms
class NormalizePAD(object):

    def __init__(self, max_size, PAD_type='right'):
        self.toTensor = transforms.ToTensor()
        self.max_size = max_size
        self.max_width_half = math.floor(max_size[2] / 2)
        self.PAD_type = PAD_type

    def __call__(self, img):
        img = self.toTensor(img)
        img.sub_(0.5).div_(0.5)
        c, h, w = img.size()
        Pad_img = torch.FloatTensor(*self.max_size).fill_(0)
        Pad_img[:, :, :w] = img  # right pad
        if self.max_size[2] != w:  # add border Pad
            Pad_img[:, :, w:] = img[:, :, w - 1].unsqueeze(2).expand(c, h, self.max_size[2] - w)

        return Pad_img

In [45]:
import math
class AlignCollate(object):

    def __init__(self, imgH=64, imgW=64, keep_ratio_with_pad=False, contrast_adjust = 0.):
        self.imgH = imgH
        self.imgW = imgW
        self.keep_ratio_with_pad = keep_ratio_with_pad
        self.contrast_adjust = contrast_adjust

    def __call__(self, images):

        if self.keep_ratio_with_pad:  # same concept with 'Rosetta' paper
            resized_max_w = self.imgW
            input_channel = 3 if images[0].mode == 'RGB' else 1
            transform = NormalizePAD((input_channel, self.imgH, resized_max_w))

            resized_images = []
            for image in images:
                w, h = image.size

                #### augmentation here - change contrast
                if self.contrast_adjust > 0:
                    image = np.array(image.convert("L"))
                    image = adjust_contrast_grey(image, target = self.contrast_adjust)
                    image = Image.fromarray(image, 'L')

                ratio = w / float(h)
                if math.ceil(self.imgH * ratio) > self.imgW:
                    resized_w = self.imgW
                else:
                    resized_w = math.ceil(self.imgH * ratio)

                resized_image = image.resize((resized_w, self.imgH), Image.BICUBIC)
                resized_images.append(transform(resized_image))
                # resized_image.save('./image_test/%d_test.jpg' % w)

            image_tensors = torch.cat([t.unsqueeze(0) for t in resized_images], 0)

        else:
            transform = ResizeNormalize((self.imgW, self.imgH))
            image_tensors = [transform(image) for image in images]
            image_tensors = torch.cat([t.unsqueeze(0) for t in image_tensors], 0)

        return image_tensors
    
AlignCollate_valid = AlignCollate(imgH=opt.imgH, imgW=opt.imgW, keep_ratio_with_pad=opt.PAD, contrast_adjust=opt.contrast_adjust)

In [46]:
def get_prediction(image):
    timg = AlignCollate_valid([image.convert("L")])
    preds = ocr_model(timg,"")
    preds_size = torch.IntTensor([preds.size(1)] * 1)
    _, preds_index = preds.max(2)
    preds_index = preds_index.view(-1)
    preds_str = converter.decode_greedy(preds_index.data, preds_size.data)
    return preds_str[0]

In [47]:

def show_bboxes_get_strings(image_path, bboxes,show=True):
    img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    if show:
        # Create figure and axes
        fig, ax = plt.subplots(1, figsize=(12, 9))

        # Display the image
        ax.imshow(img)
    strings = []
    # Loop over tensor and add boxes
    for box in bboxes:
        # Bounding box data
        x, y, w, h= box
        x = x - w / 2
        y = y - h / 2
        if show:
            # Create a Rectangle patch
            rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor="r", facecolor='none')
            # Add the patch to the Axes
            ax.add_patch(rect)
        cropbox = img[int(y):int(y+h),int(x):int(x+w)]
        result = reader.recognize(cropbox)
        strings.append(result)
    if show:
        plt.show()
    return [x[0][1] for x in strings]

In [48]:
def get_line_bboxes(v,ctype=None):
    texts = v["text"]
    polygon_info = []
    point_info = []
    for tinf in v["axes"]["x-axis"]["ticks"]:
        bbox_label = get_id(texts,tinf["id"])
        polygon_info.append(bbox_label)

    for tinf in v["axes"]["y-axis"]["ticks"]:
        bbox_label = get_id(texts,tinf["id"])
        polygon_info.append(bbox_label)
    return polygon_info


In [49]:
def get_bounding_boxes(bounding_boxes,inter_limit=0.5):
    pruned_boxes = []
    to_remove = set()

    for i, big_box in enumerate(bounding_boxes):
        x1, y1, w1, h1,_ = big_box
        area1 = w1 * h1
        for j, small_box in enumerate(bounding_boxes):
            if i != j and j not in to_remove:
                x2, y2, w2, h2,_ = small_box
                area2 = w2 * h2

                # calculate the intersection rectangle
                x_int = max(x1 - w1/2, x2 - w2/2)
                y_int = max(y1 - h1/2, y2 - h2/2)
                w_int = min(x1 + w1/2, x2 + w2/2) - x_int
                h_int = min(y1 + h1/2, y2 + h2/2) - y_int

                if w_int < 0 or h_int < 0:
                    # no overlap
                    continue

                intersection = w_int * h_int
                iou = intersection / area2  # change here to check how much of the smaller box is in the larger one

                # if smaller box is contained in larger box 80% or more
                if iou >= inter_limit:
                    to_remove.add(j)

    for k, box in enumerate(bounding_boxes):
        if k not in to_remove:
            pruned_boxes.append(box)

    return pruned_boxes


In [50]:
def iou(box1, box2):
    """Calculate Intersection Over Union (IOU) of two bounding boxes."""
    x1, y1, w1, h1, _ = box1
    x2, y2, w2, h2, _ = box2
    
    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1+w1, x2+w2)
    yi2 = min(y1+h1, y2+h2)
    inter_area = max(xi2 - xi1, 0) * max(yi2 - yi1, 0)

    box1_area = w1 * h1
    box2_area = w2 * h2
    union_area = box1_area + box2_area - inter_area
    
    iou = inter_area / union_area
    return iou

In [51]:
import warnings
warnings.filterwarnings("ignore")

In [52]:
extracted_images = []
generated_images = []
for pth in tqdm(all_imgs):
    jsonname = os.path.split(pth)[-1].replace(".jpg",".json")
    jsonname = os.path.join("../datasets/dataset/train/annotations",jsonname)
    annot = json.load(open(jsonname))
    if annot["source"]=="generated":
        generated_images.append(pth)
    else:
        extracted_images.append(pth)
len(extracted_images),len(generated_images)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 223/223 [00:00<00:00, 3543.31it/s]


(223, 0)

In [53]:
def plot_boxes(image_path, tensor,classes,confid):
    # Load image
    img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    print(img.shape)
    # Colors for different classes
    colors = ['r', 'g', 'b', 'y', 'm', 'c', 'k', 'lime', 'teal', 'purple']

    # Create figure and axes
    fig, ax = plt.subplots(1, figsize=(12, 9))

    # Display the image
    ax.imshow(img)

    # Loop over tensor and add boxes
    for i in range(tensor.shape[0]):
        # Bounding box data
        x, y, w, h=[int(x) for x in tensor[i]]
        print(x,y,w,h)
        x = x - w / 2
        y = y - h / 2
        conf, cls =confid[i],int(classes[i])
        # Create a Rectangle patch
        rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor=colors[int(cls)], facecolor='none')
        # Add the patch to the Axes
        ax.add_patch(rect)
        # Add text (confidence score)
        plt.text(x, y, str(round(conf.item(), 2)), color=colors[int(cls)], fontsize=10, 
                 horizontalalignment='left', verticalalignment='top', 
                 bbox={'facecolor': colors[int(cls)], 'alpha': 0.4, 'pad': 1})
    plt.savefig("output.png", bbox_inches='tight', pad_inches=0)
    plt.show()

In [54]:
CLASS_NAMES = ["x-label","y-label","diagram-tick",'line','dot','vertical_bar','scatter','horizontal_bar']

In [55]:
def euclidean_distance(p1,p2):
    x1,y1 = p1
    x2,y2 = p2
    return np.sqrt((x1-x2)**2+(y1-y2)**2)

In [56]:
import re
def is_number(s: str):
    s = re.sub('[,$% ]', '', s)
    try:
        float(s)
        return True
    except ValueError:
        return False
def get_number(s):
    if type(s)==str:
        return float(re.sub('[,$% ]', '', s))
    else:
        return float(s)


In [57]:
import math
def scatter_inverse(result, x_tickinfo, y_tickinfo,charttype):
    x_labels_numerical = all(is_number(label) for label in x_tickinfo.keys())
    y_labels_numerical = all(is_number(label) for label in y_tickinfo.keys())

    if x_labels_numerical:
        x_ticks = {get_number(k): v for k, v in x_tickinfo.items()}
        x_values = list(x_ticks.keys())
        x_coords = list(x_ticks.values())
    else:
        x_mapping = {info['x']: label for label, info in x_tickinfo.items()}

    if y_labels_numerical:
        y_ticks = {get_number(k): v for k, v in y_tickinfo.items()}
        y_values = list(y_ticks.keys())
        y_coords = list(y_ticks.values())
    else:
        y_mapping = {info['y']: label for label, info in y_tickinfo.items()}

    dataseries = []
    for point in result:
        x_coord, y_coord = point

        if x_labels_numerical:
            for i in range(len(x_values) - 1):
                if x_coords[i+1]['x'] <= x_coord <= x_coords[i]['x']:
                    ratio = (x_coord - x_coords[i]['x']) / (x_coords[i + 1]['x'] - x_coords[i]['x'])
                    x_val = x_values[i] + ratio * (x_values[i + 1] - x_values[i])
                    break
            else:
                ratio = (x_coord - x_coords[0]['x']) / (x_coords[-1]['x'] - x_coords[0]['x'])
                x_val = x_values[0] + ratio * (x_values[-1] - x_values[0])
        else:
            x_val = x_mapping[min(x_mapping.keys(), key=lambda x: abs(x - x_coord))]


        if y_labels_numerical:
            for i in range(len(y_values) - 1):
                if y_coords[i+1]['y'] <= y_coord <= y_coords[i]['y']:
                    ratio = (y_coord - y_coords[i]['y']) / (y_coords[i + 1]['y'] - y_coords[i]['y'])
                    y_val = y_values[i] + ratio * (y_values[i + 1] - y_values[i])
                    break
            else:
                ratio = (y_coord - y_coords[0]['y']) / (y_coords[-1]['y'] - y_coords[0]['y'])
                y_val = y_values[0] + ratio * (y_values[-1] - y_values[0])
        else:
            y_val = y_mapping[min(y_mapping.keys(), key=lambda x: abs(x - y_coord))]
            

        dataseries.append({'x': x_val, 'y': y_val})

    return dataseries

In [58]:
import math
def other_inverse(result, x_tickinfo, y_tickinfo,charttype):
    y_labels_numerical = all(is_number(label) for label in y_tickinfo.keys())
    
    x_mapping = {info['x']: label for label, info in x_tickinfo.items()}

    if y_labels_numerical:
        y_ticks = {get_number(k): v for k, v in y_tickinfo.items()}
        y_values = list(y_ticks.keys())
        y_coords = list(y_ticks.values())
    else:
        y_mapping = {info['y']: label for label, info in y_tickinfo.items()}

    dataseries = []
    for x_val,tick in x_tickinfo.items():
        x_coord, y_coord = min(result,key=lambda x: abs(x[0]-tick["x"]))
        if y_labels_numerical:
            for i in range(len(y_values) - 1):
                if y_coords[i+1]['y'] <= y_coord <= y_coords[i]['y']:
                    ratio = (y_coord - y_coords[i]['y']) / (y_coords[i + 1]['y'] - y_coords[i]['y'])
                    y_val = y_values[i] + ratio * (y_values[i + 1] - y_values[i])
                    break
            else:
                ratio = (y_coord - y_coords[0]['y']) / (y_coords[-1]['y'] - y_coords[0]['y'])
                y_val = y_values[0] + ratio * (y_values[-1] - y_values[0])
        else:
            y_val = y_mapping[min(y_mapping.keys(), key=lambda x: abs(x - y_coord))]
            
        dataseries.append({'x': x_val, 'y': y_val})
    return dataseries

def horizontalbar_inverse(result, x_tickinfo, y_tickinfo,charttype):
    y_labels_numerical = all(is_number(label) for label in y_tickinfo.keys())
    
    x_mapping = {info['x']: label for label, info in x_tickinfo.items()}

    if y_labels_numerical:
        y_ticks = {get_number(k): v for k, v in y_tickinfo.items()}
        y_values = list(y_ticks.keys())
        y_coords = list(y_ticks.values())
    else:
        y_mapping = {info['y']: label for label, info in y_tickinfo.items()}

    dataseries = []
    for x_val,tick in x_tickinfo.items():
        y_coord, x_coord = min(result,key=lambda x: abs(x[1]-tick["y"]))
        if y_labels_numerical:
            for i in range(len(y_values) - 1):
                if y_coords[i]['x'] <= y_coord <= y_coords[i+1]['x']:
                    ratio = (y_coord - y_coords[i]['x']) / (y_coords[i + 1]['x'] - y_coords[i]['x'])
                    y_val = y_values[i] + ratio * (y_values[i + 1] - y_values[i])
                    break
            else:
                ratio = (y_coord - y_coords[0]['x']) / (y_coords[-1]['x'] - y_coords[0]['x'])
                y_val = y_values[0] + ratio * (y_values[-1] - y_values[0])
        else:
            y_val = y_mapping[min(y_mapping.keys(), key=lambda x: abs(x - y_coord))]
            
        dataseries.append({'x': x_val, 'y': y_val})
    return dataseries

In [59]:
def get_paired_bbox_ticks(bboxes,bbox,idx):
    x,y,w,h = bbox
    tickbox = {}
    tcount = {}
    for x1,y1,text in bboxes:
        if text in tickbox:
            tcount[text] +=1
            text = text+str(tcount[text])
        if idx==0:
            tickbox[text] = {"x":x1,"y":y+h/2}
        else:
            tickbox[text] = {"x":x-w/2,"y":y1}
        if text not in tcount:
            tcount[text] = 0
    return tickbox
    
def get_predictions_others(classwise_preds,img,charttype,bbox):
    x_bboxes = []
    y_bboxes = []
    for x,y,w,h,_ in classwise_preds[0]:
        x1 = x - w / 2
        y1 = y - h / 2
        cropbox = img[int(y1):int(y1+h),int(x1):int(x1+w)]
        result = get_prediction(Image.fromarray(cropbox))
        x_bboxes.append([x,y,result])
    for x,y,w,h,_ in classwise_preds[1]:
        x1 = x - w / 2
        y1 = y - h / 2
        cropbox = img[int(y1):int(y1+h),int(x1):int(x1+w)]
        result = get_prediction(Image.fromarray(cropbox))
        y_bboxes.append([x,y,result])
    if charttype=="horizontal_bar":
        xidx,yidx=1,0
    else:
        xidx,yidx=0,1
    x_tickinfo = get_paired_bbox_ticks(x_bboxes,bbox,xidx)
    y_tickinfo = get_paired_bbox_ticks(y_bboxes,bbox,yidx)
    print("xtick:",len(x_tickinfo))
    print("ytick:",len(y_tickinfo))
    
    results = [tmp[:2] for tmp in classwise_preds[2]]
    
    if charttype=="horizontal_bar":
        results = sorted(results,key=lambda ttmp: ttmp[1])
        y_tickinfo = {k: v for k, v in sorted(y_tickinfo.items(), key=lambda item: item[1]["x"])}
        x_tickinfo = {k: v for k, v in sorted(x_tickinfo.items(), key=lambda item: item[1]["y"])}        
    else:
        results = sorted(results,key=lambda ttmp: ttmp[0])
        x_tickinfo = {k: v for k, v in sorted(x_tickinfo.items(), key=lambda item: item[1]["x"])}
        y_tickinfo = {k: v for k, v in sorted(y_tickinfo.items(),reverse=True, key=lambda item: item[1]["y"])}
#     print(x_tickinfo)
    if charttype=="scatter":
        answer = scatter_inverse(results,x_tickinfo,y_tickinfo,charttype)
    elif charttype=="horizontal_bar":
        answer = horizontalbar_inverse(results,x_tickinfo,y_tickinfo,charttype)
    else:
        answer = other_inverse(results,x_tickinfo,y_tickinfo,charttype)
    return answer,charttype

In [60]:
def remove_contained_boxes(boxes,THRES=0.95):
    to_remove = []
    for i in range(len(boxes)):
        for j in range(i + 1, len(boxes)):
            box1 = boxes[i]
            box2 = boxes[j]

            # Calculate the top-left and bottom-right coordinates of each box
            box1_coords = [(box1[0] - box1[2]/2, box1[1] - box1[3]/2), 
                           (box1[0] + box1[2]/2, box1[1] + box1[3]/2)]
            box2_coords = [(box2[0] - box2[2]/2, box2[1] - box2[3]/2), 
                           (box2[0] + box2[2]/2, box2[1] + box2[3]/2)]

            # Calculate the overlap of the two boxes
            dx = min(box1_coords[1][0], box2_coords[1][0]) - max(box1_coords[0][0], box2_coords[0][0])
            dy = min(box1_coords[1][1], box2_coords[1][1]) - max(box1_coords[0][1], box2_coords[0][1])
            if (dx>=0) and (dy>=0):
                overlap_area = dx*dy
                box1_area = box1[2]*box1[3]
                box2_area = box2[2]*box2[3]
                
                # If box1 is contained within box2
                if overlap_area >= THRES * box1_area:
                    to_remove.append(i)
                
                # If box2 is contained within box1
                elif overlap_area >= THRES * box2_area:
                    to_remove.append(j)

    # Remove the boxes that are contained within others
    return [box for i, box in enumerate(boxes) if i not in to_remove]

In [61]:
def get_predictions(classwise_preds,img):
    scores = []
    bbox = []
    for i in range(3,8):
        if len(classwise_preds[i])>0:
            mostconf = max(classwise_preds[i],key = lambda x: x[-1])
            scores.append(mostconf[-1])
            bbox.append(mostconf[:4])
        else:
            scores.append(0)
            bbox.append([])
            
    amx = np.argmax(scores)
    if scores[amx]==0:
        return None
    charttype = CLASS_NAMES[3+amx]
    bounding_box = bbox[amx]
    x,y,w,h = bounding_box
    xe = x+w/2
    ye = y+h/2
    x = x-w/2
    y= y-h/2
    yaxisidx,xaxisidx=1,0
    if charttype=="horizontal_bar":
        yaxisidx,xaxisidx=0,1
    # Check1: Only include diagram ticks that are inside the bounding box
    filtered = []
    for idx, (x1, y1, w1, h1,conf) in enumerate(classwise_preds[2]):
        if not ( x>(x1+w1/2) or y>(y1+h1/2) or xe<(x1-w1/2) or ye<(y1-h1/2)):
            filtered.append([x1, y1, w1, h1,conf])
    print(len(classwise_preds[2]),len(filtered))
    classwise_preds[2] = remove_contained_boxes(filtered)
    # CHeck2: Only keep yaxis labels on the left side of bounding box
    filtered = []
    for idx,(x1,y1,w1,h1,conf) in enumerate(classwise_preds[yaxisidx]):
        if (x1-w1/2)<x and (y1-h1/2)<ye and (y1+h1/2)>y:
            filtered.append([x1, y1, w1, h1,conf])
    print(len(classwise_preds[yaxisidx]),len(filtered))
    classwise_preds[yaxisidx] = remove_contained_boxes(filtered)
    print("removed contained:",len(classwise_preds[yaxisidx]),len(filtered))
    
    # Check 3: Only keep xaxis labels on the bottom side of bounding box
    filtered = []
    for idx,(x1,y1,w1,h1,conf) in enumerate(classwise_preds[xaxisidx]):
        if (y1+h1/2)>ye and (x1-w1/2)<xe and (x1+w1/2)>x:
            filtered.append([x1, y1, w1, h1,conf])
    print(len(classwise_preds[xaxisidx]),len(filtered))
    classwise_preds[xaxisidx] = remove_contained_boxes(filtered)
    print("removed contained:",len(classwise_preds[xaxisidx]),len(filtered))
#     print(classwise_preds)
    return get_predictions_others(classwise_preds,img,charttype,bounding_box)

In [62]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rapidfuzz.distance.Levenshtein import distance as levenshtein

def sigmoid(x):
    return 2 - 2 / (1 + np.exp(-x))

def r2_score(y_true, y_pred):
    ss_res = np.sum(np.square(y_true - y_pred))
    ss_tot = np.sum(np.square(y_true - np.mean(y_true)))
    r2 = 1 - (ss_res / ss_tot)
    return r2

def normalized_rmse(y_true, y_pred):
    # The argument to the sigmoid transform is equal to 
    # rmse(y_true, y_pred) / rmse(y_true, np.mean(y_true))
    return sigmoid((1 - r2_score(np.array(y_true), np.array(y_pred))) ** 0.5)


def normalized_levenshtein_score(y_true, y_pred):
    total_distance = np.sum([levenshtein(yt, yp) for yt, yp in zip(y_true, y_pred)])
    length_sum = np.sum([len(yt) for yt in y_true])
    return sigmoid(total_distance / length_sum)


def score_series(y_true, y_pred):
    y_pred = y_pred.split(";")
    if len(y_true) != len(y_pred):
        # print("unequal")
        return 0.0
    if isinstance(y_true[0], str):
        return normalized_levenshtein_score(y_true, y_pred)
    else:
        try:
            y_pred = [float(x) for x in y_pred]
        except:
            # "Error"
            return 0
        return normalized_rmse(y_true, y_pred)


def benetech_score(ground_truth: pd.DataFrame, predictions: pd.DataFrame) -> float:
    """Evaluate predictions using the metric from the Benetech - Making Graphs Accessible.
    
    Parameters
    ----------
    ground_truth: pd.DataFrame
        Has columns `[data_series, chart_type]` and an index `id`. Values in `data_series` 
        should be either arrays of floats or arrays of strings.
    
    predictions: pd.DataFrame
    """
    if not ground_truth.index.equals(predictions.index):
        raise ValueError("Must have exactly one prediction for each ground-truth instance.")
    if not ground_truth.columns.equals(predictions.columns):
        raise ValueError(f"Predictions must have columns: {ground_truth.columns}.")
    pairs = zip(ground_truth.itertuples(index=False), predictions.itertuples(index=False))
    scores = []
    for (gt_series, gt_type), (pred_series, pred_type) in pairs:
        if gt_type != pred_type:  # Check chart_type condition
            print("different charts")
            scores.append(0.0)
        else:  # Score with RMSE or Levenshtein as appropriate
            scores.append(score_series(gt_series, pred_series))
    return scores

In [63]:
# pth = random.choice(extracted_images)
# img = cv2.cvtColor(cv2.imread(pth), cv2.COLOR_BGR2RGB)
# results = detect_yolo(pth)
# tmppred = [results[:,:4].cpu(),results[:,4].cpu(),results[:,5].cpu()]    
# byclass = divide_by_class(*tmppred)
# # preds,ctype =get_predictions(byclass,img)
# jsonname = os.path.split(pth)[-1].replace(".jpg","")
# jsonpath = os.path.join("../datasets/dataset/train/annotations",jsonname+".json")
# annot = json.load(open(jsonpath))
# # print(ctype)
# # print(len(preds),len(annot["data-series"]))
# # print("x:", ([pp["x"] for pp in preds]))
# # print("y:", ([pp["y"] for pp in preds]))
# # print("gt_x:", [pp["x"] for pp in annot["data-series"] if "x" in pp])
# # print("gt_y:", [pp["y"] for pp in annot["data-series"] if "y" in pp])
# # print(len(byclass[0]))
# # plot_boxes(pth,[],[],[])
# plot_boxes(pth,results[:,:4].cpu().numpy(),results[:,4].cpu().numpy(),results[:,5].cpu())

In [64]:
ST= time.time()
predictions = {}
groundtruth = {}
ec=0
for pth in tqdm(random.sample(extracted_images,len(extracted_images))):
    img = cv2.cvtColor(cv2.imread(pth), cv2.COLOR_BGR2RGB)
    results = detect_yolo(pth)
    tmppred = [results[:,:4].cpu(),results[:,5].cpu(),results[:,4].cpu()]    
    byclass = divide_by_class(*tmppred)
    try:
        preds,ctype =get_predictions(byclass,img)
    except:
        print("Error")
        preds =[]
        ctype = "scatter"
        ec+=1
    jsonname = os.path.split(pth)[-1].replace(".jpg","")
    jsonpath = os.path.join("../datasets/dataset/train/annotations",jsonname+".json")
    annot = json.load(open(jsonpath))
    predictions[f"{jsonname}_x"] = (";".join([str(pp["x"]) for pp in preds]),ctype)
    predictions[f"{jsonname}_y"] = (";".join([str(pp["y"]) for pp in preds]),ctype)
    groundtruth[f"{jsonname}_x"] = ([pp["x"] for pp in annot["data-series"] if "x" in pp],annot["chart-type"])
    groundtruth[f"{jsonname}_y"] = ([pp["y"] for pp in annot["data-series"] if "y" in pp],annot["chart-type"])
total = time.time() - ST

  0%|                                                                                                                                      | 0/223 [00:00<?, ?it/s]

5 5
9 9
removed contained: 9 9
5 5
removed contained: 5 5


  0%|▌                                                                                                                             | 1/223 [00:02<09:18,  2.51s/it]

xtick: 5
ytick: 9
15 15
5 5
removed contained: 5 5
13 13
removed contained: 13 13


  1%|█▏                                                                                                                            | 2/223 [00:05<10:51,  2.95s/it]

...

xtick: 9
ytick: 7
22 22
8 8
removed contained: 8 8
22 22
removed contained: 22 22


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍| 222/223 [05:23<00:01,  1.98s/it]

xtick: 22
ytick: 8
3 3
10 10
removed contained: 10 10
3 3
removed contained: 3 3


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 223/223 [05:24<00:00,  1.46s/it]

xtick: 3
ytick: 10


In [65]:

prediction_df = pd.DataFrame.from_dict(predictions, orient='index', columns=['data_series', 'chart_type']).rename_axis('id')
groundt_df = pd.DataFrame.from_dict(groundtruth, orient='index', columns=['data_series', 'chart_type']).rename_axis('id')

In [66]:
mask = (prediction_df[0::2].data_series.apply(lambda x: len(x.split(";")))==groundt_df[0::2].data_series.apply(len))

In [67]:
mask.mean()

0.7443946188340808

In [68]:
# mispred=groundt_df[0::2][~(mask&(prediction_df[0::2].chart_type==groundt_df[0::2].chart_type))]

In [69]:
# opth = random.choice(mispred.index)
# pth = os.path.join("../datasets/dataset/keypoint/images/valid/",opth.split("_")[0]+".jpg")
# img = cv2.cvtColor(cv2.imread(pth), cv2.COLOR_BGR2RGB)
# results = yolo_model.predict(pth, save=False, imgsz=640, conf=0.2,boxes=True)
# predictions = [results[0].boxes.xywh.cpu(),results[0].boxes.cls.cpu(),results[0].boxes.conf.cpu()]
# byclass = divide_by_class(*predictions)
# preds,ctype =get_predictions(byclass,img)
# jsonname = os.path.split(pth)[-1].replace(".jpg","")
# jsonpath = os.path.join("../datasets/dataset/train/annotations",jsonname+".json")
# annot = json.load(open(jsonpath))
# # print(ctype)
# print(len(preds),len(annot["data-series"]))
# # print("x:", ([pp["x"] for pp in preds]))
# # print("y:", ([pp["y"] for pp in preds]))
# # print("gt_x:", [pp["x"] for pp in annot["data-series"] if "x" in pp])
# # print("gt_y:", [pp["y"] for pp in annot["data-series"] if "y" in pp])
# # print(len(byclass[0]))
# plot_boxes(pth,results[0].boxes.xywh.cpu(),results[0].boxes.cls.cpu(),results[0].boxes.conf.cpu())

In [70]:
scores = benetech_score(groundt_df, prediction_df)

unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
different charts
different charts
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
different charts
different charts
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
unequal
different charts
different charts
unequal
unequal


In [71]:
np.mean([x for x in scores if np.isfinite(x)])

0.6166933124997266